## Build Environment

In [ ]:
# If running on Vast.AI, copy the code in this cell into a new notebook. Run it, then launch the `dreambooth_runpod_joepenna.ipynb` notebook from the jupyter interface.
#!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion

In [1]:
# Download all the goodies you will need
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install captionizer==1.0.1
!apt-get update --assume-yes
!apt-get upgrade --assume-yes
!apt-get install zip --assume-yes
!apt-get install unzip --assume-yes
!apt-get install -y zip unzip --assume-yes


Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
  Updating ./src/taming-transformers clone (to revision master)
  Running command git fetch -q --tags
  Running command git reset --hard -q 3ba01b241669f5ade541ce990f7650a3b8f65318
  Preparing metadata (setup.py) ... done
  Attempting uninstall: taming-transformers
    Found existing installation: taming-transformers 0.0.1
    Uninstalling taming-transformers-0.0.1:
      Successfully uninstalled taming-transformers-0.0.1
  Running setup.py develop for taming-transformers
Obtaining clip from git+https://github.com/openai/CLIP.git@main#egg=clip
  Updating ./src/clip clone (to revision main)
  Running command git fetch -q --tags
  Running command git reset --hard -q d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
  Preparing metadata (setup.py) ... done
  Attempting uninstall: clip
    Found existing installation: clip 1.0
    Uninstalling clip-1.0:
      Successfully uninsta

In [14]:
# Hugging Face Login
from huggingface_hub import notebook_login

notebook_login()

In [1]:
#Download Reggies, using these by default is highly recommended
hf_hub_download(repo_id="senseiberia/768_regularization_images", filename="768_person_eulera_80steps_cfg5.zip", repo_type="dataset") '/workspace/Dreambooth-Stable-Diffusion'

#Download the 2.1 model next
#hf_hub_download(repo_id="stabilityai/stable-diffusion-2-1", filename="v2-1_768-ema-pruned.ckpt") '/workspace/Dreambooth-Stable-Diffusion'


Downloading: 100%|███████████████████████████| 966M/966M [00:10<00:00, 96.0MB/s]
Downloading: 100%|█████████████████████████| 1.39G/1.39G [00:15<00:00, 88.0MB/s]


All good G. All good. Check the regularization_images/master folder.


In [24]:
#Unzip the Reggies into its folder

!unzip /root/.cache/huggingface/hub/datasets--ProGamerGov--StableDiffusion-v1-5-Regularization-Images/snapshots/33c3dfd363f20ff50042f2d22252b1ec74ddaa21/person_v1-5_mse_vae_ddim50_cfg7_n2115.zip -j /workspace/Dreambooth-Stable-Diffusion/regularization_images

In [ ]:
#Unzip your training images into its folder

!unzip /workspace/Dreambooth-Stable-Diffusion/zippy.zip -j /workspace/Dreambooth-Stable-Diffusion/training_images

## Training

Two sets of images will be generated, sample_subject and sample, use these to keep track of your training progress. They will be stored in the logs/images/train folder. Don't forget to change your training settings by editing the trainingsettings.yaml file!

Also, make sure to edit the model name if you are using something other than the 768-v-ema ckpt which is the 2.1 model for 768x768 training.


In [ ]:
# Training

# This isn't used for training, just to help you remember what your trained into the model.
project_name = "xmeetsultimate"

# MAX STEPS
# How many steps do you want to train for?
max_training_steps = 13000

# Match class_word to the category of the regularization images you chose above.
class_word = "person" # typical uses are "man", "person", "woman"

# This is the unique token you are incorporating into the stable diffusion model.
token = "xmeetsultimate"


reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/regularization_images/"

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --base /workspace/Dreambooth-Stable-Diffusion/trainingsettings.yaml \
 -t \
 --actual_resume "v2-1_768-ema-pruned.ckpt" \
 --reg_data_root "{reg_data_root}" \
 -n "{project_name}" \
 --gpus 0, \
 --data_root "/workspace/Dreambooth-Stable-Diffusion/training_images" \
 --max_training_steps {max_training_steps} \
 --class_word "{class_word}" \
 --token "{token}" \
 --no-test

## Copy and name the checkpoint file

In [46]:
# After the training is done just run this, then go to the trained_models folder which will be created, go ahead and stop (NOT terminate) this runpod so you can access this without GPU to avoid extra costs and just download the ckpt file from there. Voila! You're done!

directory_paths = !ls -d logs/*
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_images)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + token + "_token_" + class_word + "_class_word.ckpt"

file_name = file_name.replace(" ", "_")

!mkdir -p trained_models
!mv "{last_checkpoint_file}" "trained_models/{file_name}"

print("Download your trained model file from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")

Download your trained model file from trained_models/2022-12-18T16-00-29_gabimeetsxxx1_429_training_images_43430_max_training_steps_gabimeetsxxx1_token_woman_class_word.ckpt and use in your favorite Stable Diffusion repo!


# Optional - Upload to google drive
* run the following commands in a new `terminal` in the `Dreambooth-Stable-Diffusion` directory
* `chmod +x ./gdrive`
* `./gdrive about`
* `paste your token here after navigating to the link`
* `./gdrive upload trained_models/{file_name.ckpt}`

# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`

## Generate Images With Your Trained Model!

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter 4 \
 --scale 7.0 \
 --ddim_steps 50 \
 --ckpt "/workspace/Dreambooth-Stable-Diffusion/trained_models/{file_name}" \
 --prompt "joepenna person as a masterpiece portrait painting by John Singer Sargent in the style of Rembrandt"